In [1]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from xgboost import XGBRFClassifier
import numpy as np
import pandas as pd

In [15]:
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()
df = df[(df.league=="Barclays Premier League") | (df.league=="UEFA Champions League")]

In [16]:
df['h_margin'] = df['score1'] - df['score2']

In [17]:
results = []

In [18]:
for x in df['h_margin']:
  if x > 0:
    results.append("HOME WIN")
  elif x < 0:
    results.append("AWAY WIN")
  else:
    results.append("DRAW")

In [19]:
df = df.drop(columns=['h_margin'])
df['result'] = results

In [22]:
X = df[['score1','score2']]
y = df['result']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [23]:
model = XGBRFClassifier(max_depth=6, objective='multi:softmax', n_estimators=400)

In [24]:
model.fit(X_train, y_train)

XGBRFClassifier(base_score=0.5, colsample_bylevel=1, colsample_bynode=0.8,
                colsample_bytree=1, gamma=0, learning_rate=1, max_delta_step=0,
                max_depth=6, min_child_weight=1, missing=None, n_estimators=400,
                n_jobs=1, nthread=None, objective='multi:softprob',
                random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                seed=None, silent=None, subsample=0.8, verbosity=1)

In [25]:
predictions = model.predict(X_test)

In [26]:
accuracy = accuracy_score(y_test, predictions)

In [29]:
print("Test Accuracy: {}%".format(np.round(accuracy * 100),3))

Test Accuracy: 100.0%


In [30]:
kfold = KFold(n_splits=10, shuffle=True)

In [31]:
cv_score = cross_val_score(model, X, y, cv=kfold)

In [33]:
cv_avg = np.mean(cv_score)

In [34]:
print("Cross-Validation Mean: {}".format(np.round(cv_avg),3))

Cross-Validation Mean: 1.0
